In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
# 데이터 읽기
df = pd.read_csv(url)
print(df.head())
print("\n=====================================================\n")

# 결측치 확인
print(df.isnull().sum())
print("\n=====================================================\n")

# 범주형 데이터를 숫자로 변환
label_encoders = {} # 각 컬럼별 LabelEncoder 객체를 저장할 딕셔너리 생성
columns = ['sex', 'smoker', 'region']
for column in columns:
  label_encoders[column] = LabelEncoder() # 해당 컬럼에 대해 LabelEncoder 객체 생성
  df[column] = label_encoders[column].fit_transform(df[column]) # 데이터를 숫자로 변환

X = df.drop(['charges'], axis=1).values # Feature
Y = df['charges'].values.astype(np.float32) # 예측해야 할 column

# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# numpy 배열 --> torch tensor으로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).view(-1, 1) # view(-1, 1) : (행, 1) 형태로 변환. 모델이 기대하는 입력 형태는 2차원 배열이기 때문이다.
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32).view(-1, 1)

# torch tensor 데이터의 shape을 출력
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print("\n=====================================================\n")

# PyTorch DataLoader로 감싸서 미니배치 학습 가능하게 함
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # 배치 사이즈는 32로 설정.
test_loader = DataLoader(test_dataset, batch_size=32)

# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential( # 층들을 나열
            nn.Linear(6, 64), # 입력층 : 특성 개수 6
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1) # 회귀 : 출력 뉴런 수 1
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # GPU가 있으면 GPU 사용, 아니면 CPU 사용한다는 뜻
model = RegressionModel().to(device) # 모델 생성
criterion = nn.MSELoss() # 회귀의 손실 함수 : MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 : Adam

# 학습 루프
model.train() # 학습 모드
for epoch in range(50): # 50 epoch동안 훈련
    total_loss = 0 # loss의 초기값은 당연히 0으로 설정
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch) # 배치별로 예측 수행. output : 모델이 예측한 실제 수치값(회귀)
        loss = criterion(output, Y_batch) # 예측값(output)과 실제 값(Y_batch)을 매치하여 loss 계산
        loss.backward() # 역방향 전파로 가중치/편향 업데이트
        optimizer.step()
        total_loss += loss.item() # loss를 누적
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}") # epoch마다 loss를 출력

# 평가
model.eval() # 평가 모드
preds, actuals = [], []
with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy() # 예측값을 numpy 형태로 변환 후,
        preds.extend(outputs) # preds 리스트에 예측값을 저장
        actuals.extend(Y_batch.numpy()) # actuals 리스트에 실제 값을 저장

mse = mean_squared_error(actuals, preds) # 실제 값과 예측 값의 mse를 계산하고 출력.
print(f"Test MSE: {mse:.4f}")

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


torch.Size([1070, 6])
torch.Size([1070, 1])
torch.Size([268, 6])
torch.Size([268, 1])


Epoch 1, Loss: 931.9937
Epoch 2, Loss: 813.3646
Epoch 3, Loss: 580.7716
Epoch 4, Loss: 286.7850
Epoch 5, Loss: 92.8996
Epoch 6, Loss: 48.3390
Epoch 7, Loss: 44.1162
Epoch 8, Loss: 42.2910
Epoch 9, Loss: 41.2371
Epoch 10, Loss: 39.5107
Epoch 11, Loss: 38.5242
Epoch 12, Loss: 37.6379
Epoch 13, Loss: 37.1260
Epoch 14, Loss: 36.1929
Epoch 15, Loss: 35.6336
Epoch 16, Loss: 34.9911
Epoch 17, Loss: 34.4530
Epoch 18, 